# Main Training Book for Our PCFG

In [1]:
# Import dependencies
import torch

import pcfg_models as pcfg_models
import pcfg_loader as loader
from synthetic_generator import get_dsl_operations
import source as source
import image_encoder as encoder

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

Device: cuda


In [2]:
# Import our data yielder
data_folder = 'training'
loader_params = {
    'batch_size': 1,
    'shuffle': True,
    'only_inputs': True,
    'print_steps': True,
    'moves_per_step': 3,
    'max_steps': 3,
    'p_use_base': 0.25,
}

data_yielder = loader.get_pcfg_datayielder(data_folder, loader_params)

In [3]:
# Import our embedding model
embedding_model = source.embedding.load_ViT(device)

C:\Users\lucas\Desktop\UCSD\291C - Program Synthesis\Project\ARC-AGI\image_encoder\transformers\vision_transformer.py:275: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  chec

Vision Transformer instantiated with 398,144 parameters using Sinusoidal encodings.


In [4]:
# Instantiate our Encoder classifier
dsl_list = data_yielder.label_ops
pcfg_encoder = pcfg_models.PCFG_Encoder(
    n_embd = embedding_model.model_params['embed_dim'], 
    n_head = 4, 
    n_layer = 6, 
    ff_hd = int(embedding_model.model_params['embed_dim'] * 4), 
    dropout = 0.2, 
    block_size = 33, 
    dsl_mapping = dsl_list, 
    embedding_model = embedding_model, 
    device = device
)

Vision Transformer instantiated with 699,392 parameters.


In [ ]:
# Loop through data
print_samples = loader_params['print_steps']

for i in range(20):
    try:
        key, x, y, prev_idx = next(data_yielder)
    except StopIteration as e:
        continue
    
    if print_samples:
        print(f"Key: {key}")
        print(f"x:   {x}")
        print(f"y:   {y}")
        print(f"Obj idx previous step applied to: {prev_idx}")
        print("\n")